Word2Vec

   - 분류 등과 같이 별도의 레이블이 없이 텍스트 자체만 있어도 학습이 가능
   - Word2Vec의 방식은 주변 관계를 이용하는 것으로 2가지의 방식이 있음
   - 1. CBOW(continuous Bag-of-words) : 주변 단어의 임베딩을 더해서 대상단어를 예측
   - 2. Skip-Gram : 대상 단어의 임베딩으로 주변단어를 예측
      - 일반적으로 CBOW보다 성능이 좋은 편이지만 한번에 여러 단어를 예측해야하기 때문에 비효율적
      - 최근에는 negative sampling이라는 방법을 사용

T-SNE(t-Stochastic Neighbor Embedding)
   - T-SNE은 고차원의 벡터들의 구조를 보존하며 저차원으로 사상하는 차원 축소 알고리즘
   - 단어 임베딩에서도 생성된 고차원 벡터들을 시각화하기 위해 T-SNE 알고이즘을 많이 이용
   - t-sne는 가장 먼저 원 공간의 데이터 유사도와 임베딩 공간의 데이터 유사도를 정의

In [3]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, 
                             remove=('headers', 'footers', 'quotes'))   # word2vec하는거라 필요없는 것들을 제거

documents = dataset.data

print(len(documents))

11314


In [4]:
import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def clean_text(d):
    pattern = r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', d)
    return d

# 불용어 제거
def clean_stopword(d):
    stop_words = stopwords.words('english')
    return ' '.join([w.lower() for w in d.split() if w not in stop_words and len(w) > 3])

def tokenize(d):
    return word_tokenize(d)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bitcamp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bitcamp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Pandas Frame으로 변환
import pandas as pd
news_df = pd.DataFrame({'article':documents})

news_df.replace("", float("NaN"), inplace=True)
news_df.dropna(inplace=True)
print(len(news_df))

11096


In [6]:
# 특수문자 제거
news_df['article'] = news_df['article'].apply(clean_text)

# 불용어 제거
news_df['article'] = news_df['article'].apply(clean_stopword)

# 토크나이저
tokenized_news = news_df['article'].apply(tokenize)
tokenized_news = tokenized_news.to_list()

In [7]:
import numpy as np

drop_news = [index for index, sentence in enumerate(tokenized_news) if len(sentence) <= 1]
news_texts = np.delete(tokenized_news, drop_news, axis=0)
print(len(news_texts))

10991


C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Gensim을 이용한 Word2Vec

1. CBOW

In [8]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=news_texts, window=3, vector_size=100, min_count=5, workers=4, sg=0)

# window : 앞뒤로 몇개의 단어를 보고 유추할 것인가? 보통 3,4
# vector_size : 
# workers : 병렬로 처리
# sg : CBOW는 0, skip-gram은 0

model.wv.similarity('man', 'woman')

0.85426056

In [9]:
print(model.wv.most_similar(positive=['soldiers']))
print(model.wv.most_similar(positive=['man', 'soldiers'], negative=['woman']))

[('killed', 0.9599592685699463), ('turks', 0.9569918513298035), ('villages', 0.9568962454795837), ('genocide', 0.9529709815979004), ('armenia', 0.9516713619232178), ('troops', 0.9478585124015808), ('began', 0.9359217286109924), ('land', 0.9322299957275391), ('greece', 0.9317634701728821), ('arms', 0.9315558075904846)]
[('jews', 0.9218937158584595), ('war', 0.9064255356788635), ('land', 0.8951766490936279), ('turkey', 0.8849934935569763), ('men', 0.8849554657936096), ('israel', 0.8801628947257996), ('attack', 0.8769087195396423), ('israelis', 0.8650898933410645), ('peace', 0.8613511323928833), ('murder', 0.8612600564956665)]


2. Skip-gram

In [10]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=news_texts, window=3, vector_size=100, min_count=5, workers=4, sg=1)
# sg=1로만 변경하면 됨

model.wv.similarity('man', 'woman')

0.8300685

In [11]:
print(model.wv.most_similar(positive=['soldiers']))
print(model.wv.most_similar(positive=['man', 'soldiers'], negative=['woman']))

[('villages', 0.9391025900840759), ('azerbaijanis', 0.9345746636390686), ('wounded', 0.931286096572876), ('azeri', 0.9190162420272827), ('kurds', 0.9106958508491516), ('village', 0.9101150035858154), ('raped', 0.908791184425354), ('murdered', 0.9070382118225098), ('burned', 0.9055033922195435), ('troops', 0.897631049156189)]
[('civilians', 0.8298513889312744), ('murders', 0.8295454382896423), ('babies', 0.823814868927002), ('arafat', 0.8204160332679749), ('murder', 0.8184711933135986), ('jew', 0.8148356080055237), ('rape', 0.8130385875701904), ('israelis', 0.8122097253799438), ('wwii', 0.8100835084915161), ('saints', 0.8093860149383545)]


임베딩 벡터 시각화

In [12]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('news_w2v')

# 벡터화된 값을 tsv 파일로 저장하기
!python -m gensim.scripts.word2vec2tensor --input news_w2v --output naver_w2v

2022-02-28 09:21:47,502 - word2vec2tensor - INFO - running C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\gensim\scripts\word2vec2tensor.py --input news_w2v --output naver_w2v
2022-02-28 09:21:47,502 - keyedvectors - INFO - loading projection weights from news_w2v
2022-02-28 09:21:48,752 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (22220, 100) matrix of type float32 from news_w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-02-28T09:21:48.752945', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'load_word2vec_format'}
2022-02-28 09:21:50,072 - word2vec2tensor - INFO - 2D tensor file saved to naver_w2v_tensor.tsv
2022-02-28 09:21:50,072 - word2vec2tensor - INFO - Tensor metadata file saved to naver_w2v_metadata.tsv
2022-02-28 09:21:50,074 - word2vec2tensor - INFO - finished running word2vec2tensor.py


Embedding Projector : https://projector.tensorflow.org/